In [1]:
from sqlalchemy import create_engine
import pandas as pd
import json
import urllib.parse
import psycopg2
import os 
with open("credentials.json") as file:
    login = json.load(file)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

engine = create_engine("postgresql://{}:{}@{}:{}/postgres".format(username,password,host,port))


In [2]:
# this script is using to establish sql database
path = "path_to_data"

files = os.listdir(path)
with engine.begin() as connection:    
    for file in files:
        df = pd.read_stata(path + file)
        df.to_sql(name = file, con = connection, index=False)
